# Multi-Cycle Time-Dependent Fitting

This notebook demonstrates how to fit data with multiple distinct sub-cycles within one pump-probe cycle.

**Example System:**
- Two distinct temporal regions with different dynamics
- Two Gaussian-Lorentzian Product (GLP) peaks
- Components: offset + Shirley background
- Technique: X-ray Photoelectron Spectroscopy (XPS)

**Use cases:**
- Sequential pump-probe excitations
- Different dynamics in different time regions
- Periodic phenomena with phase-dependent behavior
- Pump-repump-probe experiments
- Multi-pulse excitation schemes

In [ ]:
import os
import numpy as np
import trspecfit
from trspecfit.utils.lmfit import MC  # For uncertainty estimation

## 1. Load Data

Load experimental data containing multiple distinct temporal dynamics.

In [ ]:
# Create project
project = trspecfit.Project(path=os.getcwd())

In [ ]:
# Load data from CSV files
data_folder = "data"

file = trspecfit.File(
    parent_project=project,
    path=data_folder,
    data=np.loadtxt(project.path / data_folder / "data.csv", delimiter=','),
    energy=np.loadtxt(project.path / data_folder / "energy.csv"),
    time=np.loadtxt(project.path / data_folder / "time.csv")
)

print(f"Data shape: {file.data.shape}")
print(f"Energy range: {file.energy.min():.1f} - {file.energy.max():.1f}")
print(f"Time range: {file.time.min():.1f} - {file.time.max():.1f}")

## 2. Inspect Data

Identify the different temporal regions and their characteristics.

In [ ]:
# Visualize full dataset
file.describe()

## 3. Define Fitting Region

Set energy and time limits to include all relevant temporal regions.

In [ ]:
# Set fitting limits
file.set_fit_limits(
    energy_limits=[92, 81],  # Energy range in eV
    time_limits=[-1.5, 10]  # Time range covering both sub-cycles
)

## 4. Define Baseline Spectrum

Extract the pre-excitation ground state spectrum.

In [ ]:
# Define baseline using absolute time values
file.define_baseline(
    time_start=-1.5,
    time_stop=0,
    time_type='abs'  # Absolute time values
)

## 5. Fit Baseline Spectrum

Establish ground state parameters before analyzing dynamics.

In [ ]:
# Load and fit baseline model
file.load_model(
    model_yaml="models_energy.yaml",
    model_info=["base"]
)

file.describe_model(model_info="base", detail=0)

In [ ]:
# Fit baseline spectrum
file.fit_baseline(model_name="base", fit=2)

## 6. (Optional) Slice-by-Slice Fitting

Examine how parameters evolve across both temporal regions before deciding on 2D fit model.

In [ ]:
# Load Slice-by-Slice model
file.load_model(
    model_yaml="models_energy.yaml",
    model_info=["SbS"]
)

file.describe_model("SbS", detail=0)

In [ ]:
# Perform Slice-by-Slice fit
file.fit_SliceBySlice(
    model_name="SbS",
    fit=1,
    try_CI=0
)

# Examine parameter evolution to identify distinct temporal regions

## 7. Multi-Cycle Global 2D Fitting

Fit with different time-dependent models for different temporal sub-cycles.

### Key Concept:
The `model_info` list defines separate models for each sub-cycle:
- **Element 0**: Applies to ALL time points (baseline convolution, global effects)
- **Element 1+**: Apply only within their respective sub-cycles

The `frequency` parameter defines sub-cycle boundaries:
- `frequency=0.5` means sub-cycle transition at T/2
- `frequency=0.33` means transition at T/3
- Each sub-cycle gets its own time-dependent model

In [ ]:
# Load 2D model
file.load_model(
    model_yaml="models_energy.yaml",
    model_info=["2D"]
)

file.describe_model(model_info="2D", detail=0)

In [ ]:
# Add multi-cycle time dependence
file.add_time_dependence(
    model_yaml="models_time.yaml",
    model_info=[
        "ModelNone",    # No dynamics (element 0: applies everywhere)
        "MonoExpNeg",   # First sub-cycle: negative exponential decay
        "MonoExpPos"    # Second sub-cycle: positive exponential growth
    ],
    par_name="GLP_01_x0",  # Peak position varies differently in each cycle
    frequency=0.5  # Sub-cycle transition at t = T/2
)

print("\n=== Multi-Cycle Model Structure ===")
file.describe_model(model_info=["2D"], detail=1)

In [ ]:
# Configure uncertainty estimation
mc_settings = MC(
    useMC=0,
    steps=5000,
    nwalkers=20,
    thin=1
)

# Perform global 2D fit
file.fit_2Dmodel(
    model_name="2D",
    fit=2,
    try_CI=0,
    MCsettings=mc_settings
)

## Tips for Multi-Cycle Fitting

**Identifying Sub-Cycles:**
- Use Slice-by-Slice fits to identify distinct temporal regions
- Look for abrupt changes in parameter evolution
- Consider physical timing (pulse spacing, excitation schemes)
- Plot parameter vs time to visualize transitions

**Model Structure:**
- **Element 0** ("ModelNone" typical): Baseline effects present throughout
- **Element 1+**: Dynamics specific to each sub-cycle
- Can use different functional forms per sub-cycle
- Each sub-cycle independently parameterized

**Frequency Parameter:**
- Defines relative sub-cycle durations
- `frequency=0.5`: Equal duration sub-cycles
- `frequency=0.33`: First cycle 1/3 duration, second 2/3
- Set based on experimental timing or optimize if uncertain

**Common Multi-Cycle Scenarios:**
1. **Pump-Repump-Probe**: Initial excitation → relaxation → re-excitation
   - Use: `["ModelNone", "MonoExp", "MonoExp"]`
   - Different decay constants for each phase

2. **Oscillatory Dynamics**: Coherent → Incoherent transitions
   - Use: `["ModelNone", "DampedOscillation", "MonoExpDecay"]`
   - Captures dephasing transition

3. **Sequential Processes**: Forward → Reverse reactions
   - Use: `["ModelNone", "MonoExpPos", "MonoExpNeg"]`
   - Models build-up then decay

**Fitting Strategy:**
- Start with single-cycle model first
- Add sub-cycles only if residuals show distinct regions
- Compare single vs multi-cycle reduced chi-squared
- Verify sub-cycle boundaries make physical sense
- Test sensitivity to frequency parameter

**Parameter Interpretation:**
- Each sub-cycle has independent parameters
- Compare time constants between sub-cycles
- Check for consistent physical interpretation
- Validate against independent measurements

**Computational Notes:**
- Multi-cycle fits are more expensive (more parameters)
- Good initial guesses are critical
- Consider fitting sub-cycles independently first
- May require longer optimization (use fit=2)

**Model Selection:**
- Use F-test or AIC/BIC to compare models
- Balance fit quality vs parameter count
- Prefer simpler model unless significantly better fit
- Validate with held-out data if available